In [15]:
# imports
import pandas as pd
import numpy as np
from google.cloud import storage
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
import functions_2

%matplotlib inline

In [89]:
# Get files from GCS bucket
BUCKET_NAME = 'salary-data'
client = storage.Client()
bucket = client.get_bucket(BUCKET_NAME)

blobs = bucket.list_blobs()
files = []
for blob in blobs:
    files.append(blob.name)
    
print(files)

['2011_Travel_to_Work_Areas_summary_statistics_V5.csv', 'Location_Tree.csv', 'Test_rev1.csv', 'Train_rev1.csv', 'Valid_rev1.csv', 'data_location_buckets.csv', 'mean_benchmark.csv', 'random_forest_benchmark_test_rev1.csv', 'test.csv']


In [88]:
# Read data
df_TTWA = pd.read_csv('gs://{}/{}'.format(BUCKET_NAME,files[0]))
df_train = pd.read_csv('gs://{}/{}'.format(BUCKET_NAME,files[3]))

In [91]:
from functions_6 import TTWA_county_feature
#df_loc = TTWA_county_feature(df_train,df_TTWA,True) 


In [93]:
TTWA_county = df_loc.TTWA_County
df_all = pd.concat([df_train, TTWA_county], axis = 1)
df_all

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName,TTWA_County
0,12612628,Engineering Systems Analyst,Engineering Systems Analyst Dorking Surrey Sal...,"Dorking, Surrey, Surrey",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,Surrey
1,12612830,Stress Engineer Glasgow,Stress Engineer Glasgow Salary **** to **** We...,"Glasgow, Scotland, Scotland",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 35000/annum 25-35K,30000,cv-library.co.uk,Glasgow
2,12612844,Modelling and simulation analyst,Mathematical Modeller / Simulation Analyst / O...,"Hampshire, South East, South East",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 40000/annum 20-40K,30000,cv-library.co.uk,Hampshire
3,12613049,Engineering Systems Analyst / Mathematical Mod...,Engineering Systems Analyst / Mathematical Mod...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,25000 - 30000/annum 25K-30K negotiable,27500,cv-library.co.uk,Surrey
4,12613647,"Pioneer, Miser Engineering Systems Analyst","Pioneer, Miser Engineering Systems Analyst Do...","Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,20000 - 30000/annum 20-30K,25000,cv-library.co.uk,Surrey
...,...,...,...,...,...,...,...,...,...,...,...,...,...
244763,72705211,TEACHER OF SCIENCE,Position: Qualified Teacher Subject/Specialism...,Swindon,Swindon,NaN,contract,NaN,Teaching Jobs,450 - 500 per week,22800,hays.co.uk,Swindon
244764,72705212,TEACHER OF BUSINESS STUDIES AND ICT,Position: Qualified Teacher or NQT Subject/Spe...,Swindon,Swindon,NaN,contract,NaN,Teaching Jobs,450 - 500 per week,22800,hays.co.uk,Swindon
244765,72705213,ENGLISH TEACHER,Position: Qualified Teacher Subject/Specialism...,Swindon,Swindon,NaN,contract,NaN,Teaching Jobs,450 - 500 per week,22800,hays.co.uk,Swindon
244766,72705216,SUPPLY TEACHERS,Position: Qualified Teacher Subject/Specialism...,Wiltshire,Wiltshire,NaN,contract,NaN,Teaching Jobs,450 to 500 per week,22800,hays.co.uk,Wiltshire


In [94]:
client = storage.Client()
bucket = client.get_bucket(BUCKET_NAME)
bucket.blob('data_location_buckets.csv').upload_from_string(df_all.to_csv(), 'text/csv')

## Deal with NaN values

In [76]:
# Fill in missing values for Contract and Company
df_all.ContractType = df_train.ContractType.fillna('Missing')
df_all.ContractTime = df_train.ContractTime.fillna('Missing')
df_all.Company = df_train.Company.fillna('Missing')
# Drop other NaN values
df_all = df_all.dropna()
df_all.isnull().sum()

Id                    0
Title                 0
FullDescription       0
LocationRaw           0
LocationNormalized    0
ContractType          0
ContractTime          0
Company               0
Category              0
SalaryRaw             0
SalaryNormalized      0
SourceName            0
TTWA                  0
County                0
dtype: int64

,TTWA,County,TTWA_County
0,NaN,Surrey,Surrey
1,Glasgow,NaN,Glasgow
2,NaN,Hampshire,Hampshire
3,NaN,Surrey,Surrey
4,NaN,Surrey,Surrey
...,...,...,...
244763,Swindon,NaN,Swindon
244764,Swindon,NaN,Swindon
244765,Swindon,NaN,Swindon
244766,NaN,Wiltshire,Wiltshire
